In [ ]:
import wandb
import torch
import torch.nn as nn
from torch import optim
import numpy as np
import timeit
from tqdm import tqdm

import model
from Bullying10k.braincog.utils import *
from Bullying10k.Bullying10k import get_bullying10k_data

In [ ]:
def train(model, criterion, optimizer, data_loader):
    start_time = timeit.default_timer()
    running_loss = 0.0
    running_corrects = 0.0
    model.train()
    for inputs, labels in tqdm(data_loader['train']):
        inputs = inputs.to(device).float()
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)

        probs = nn.Softmax(dim=1)(outputs)
        preds = torch.max(probs, 1)[1]
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / trainval_sizes['train']
    epoch_acc = running_corrects.double() / trainval_sizes['train']
    wandb.log({"accuracy": epoch_acc, "loss": epoch_loss})
    print("[{}] Epoch: {}/{} Loss: {} Acc: {}".format('train', epoch+1, config.EPOCHS, epoch_loss, epoch_acc))
    stop_time = timeit.default_timer()
    print("Execution time: " + str(stop_time - start_time) + "\n")


In [ ]:
def validate(model, criterion, optimizer, data_loader):
    start_time = timeit.default_timer()
    running_loss = 0.0
    running_corrects = 0.0

    all_preds = []
    all_labels = []
    model.eval()

    for inputs, labels in tqdm(data_loader['val']):
        inputs = inputs.to(device).float()
        labels = labels.to(device)
        optimizer.zero_grad()
        with torch.no_grad():
            outputs = model(inputs)

        probs = nn.Softmax(dim=1)(outputs)
        preds = torch.max(probs, 1)[1]
        loss = criterion(outputs, labels)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())


        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / trainval_sizes['val']
    epoch_acc = running_corrects.double() / trainval_sizes['val']
    wandb.sklearn.plot_confusion_matrix(all_labels, all_preds, [i for i in range(8)])
    print("[{}] Epoch: {}/{} Loss: {} Acc: {}".format('val', epoch+1, config.EPOCHS, epoch_loss, epoch_acc))
    stop_time = timeit.default_timer()
    print("Execution time: " + str(stop_time - start_time) + "\n")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
wandb.login()

# C3D

In [ ]:
wandb.init(
    project="Bullying10K - C3D",
    name="Fine Tune"
)

config = wandb.config
config.BATCH_SIZE = 4
config.STEP = 30
config.GAP = 4
config.SIZE = 112
config.NUM_CLASSES = 6
config.LEARNING_RATE = 1e-5
config.EPOCHS = 30
config.SEED = 42
config.WEIGHT_DECAY = 1e-4

torch.manual_seed(config.SEED)
np.random.seed(config.SEED)
train_dataloader,test_dataloader,_,_ = get_bullying10k_data(batch_size = config.BATCH_SIZE, step = config.STEP,
                                                            gap = config.GAP, size = config.SIZE)
val_dataloader = train_dataloader
trainval_loaders = {'train': train_dataloader, 'val': val_dataloader}
trainval_sizes = {x: len(trainval_loaders[x].dataset) for x in ['train', 'val']}
test_size = len(test_dataloader.dataset)

c3d = model.C3D.C3D(num_classes = config.NUM_CLASSES, pretrained = False, inchannel = 2)
c3d.to(device)
train_params = [{'params': model.C3D.get_1x_lr_params(c3d), 'lr': config.LEARNING_RATE},
                {'params': model.C3D.get_10x_lr_params(c3d), 'lr': config.LEARNING_RATE * 10}]

criterion = nn.CrossEntropyLoss()  
optimizer = optim.Adam(train_params, lr = config.LEARNING_RATE, 
                       weight_decay = config.WEIGHT_DECAY)

wandb.watch(c3d, log="all")
for epoch in range(config.EPOCHS):
    train(c3d,criterion,optimizer,trainval_loaders)

for epoch in range(1):
    validate(c3d,criterion,optimizer,trainval_loaders)

wandb.finish()

# I3D

In [ ]:
wandb.init(
    project="Bullying10K - I3D",
    name="Testing"
)

config = wandb.config
config.BATCH_SIZE = 4
config.STEP = 30
config.GAP = 4
config.SIZE = 112
config.NUM_CLASSES = 6
config.LEARNING_RATE = 1e-5
config.EPOCHS = 30
config.SEED = 42
config.WEIGHT_DECAY = 1e-4

torch.manual_seed(config.SEED)
np.random.seed(config.SEED)
train_dataloader,test_dataloader,_,_ = get_bullying10k_data(batch_size = config.BATCH_SIZE,step = config.STEP,
                                                            gap = config.GAP, size = config.SIZE)
val_dataloader = train_dataloader
trainval_loaders = {'train': train_dataloader, 'val': val_dataloader}
trainval_sizes = {x: len(trainval_loaders[x].dataset) for x in ['train', 'val']}
test_size = len(test_dataloader.dataset)

i3d = model.I3D.Inception3D(num_classes = config.NUM_CLASSES)
i3d.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(i3d.parameters(), lr = config.LEARNING_RATE, 
                       weight_decay = config.WEIGHT_DECAY)

wandb.watch(i3d, log="all")
for epoch in range(config.EPOCHS):
    train(i3d,criterion,optimizer,trainval_loaders)

for epoch in range(1):
    validate(i3d,criterion,optimizer,trainval_loaders)
wandb.finish()


# X3D

In [ ]:
wandb.init(
    project="Bullying10K - X3D",
    name="Testing"
)

config = wandb.config
config.BATCH_SIZE = 4
config.STEP = 30
config.GAP = 4
config.SIZE = 112
config.NUM_CLASSES = 6
config.LEARNING_RATE = 1e-5
config.EPOCHS = 50
config.SEED = 42
config.WEIGHT_DECAY = 1e-4
config.LAYERS = (4, 4, 4, 4)
config.CHANNELS =(64, 256, 512,1024)
config.EXPANSION = 4

torch.manual_seed(config.SEED)
np.random.seed(config.SEED)
train_dataloader,test_dataloader,_,_ = get_bullying10k_data(batch_size = config.BATCH_SIZE, step = config.STEP,
                                                            gap = config.GAP, size = config.SIZE)
val_dataloader = train_dataloader
trainval_loaders = {'train': train_dataloader, 'val': val_dataloader}
trainval_sizes = {x: len(trainval_loaders[x].dataset) for x in ['train', 'val']}
test_size = len(test_dataloader.dataset)

x3d = model.X3D.X3D(num_classes = config.NUM_CLASSES, layers = config.LAYERS, 
                    channels=config.CHANNELS, expansion = config.EXPANSION)
x3d.to(device)
criterion = nn.CrossEntropyLoss()  
optimizer = optim.Adam(x3d.parameters(), lr = config.LEARNING_RATE, 
                       weight_decay = config.WEIGHT_DECAY)

wandb.watch(x3d, log="all")
for epoch in range(config.EPOCHS):
    train(x3d,criterion,optimizer,trainval_loaders)

for epoch in range(1):
    validate(x3d,criterion,optimizer,trainval_loaders)
wandb.finish()